In [65]:
import requests
import bs4
import re
import numpy as np
import pandas as pd

In [2]:
# Trial

In [3]:
base_url = 'https://www.n11.com/televizyon-ve-ses-sistemleri/televizyon/led-tv?pg={}'

In [4]:
base_url.format('50')

'https://www.n11.com/televizyon-ve-ses-sistemleri/televizyon/led-tv?pg=50'

In [5]:
# There are 50 pages.

In [6]:
res = requests.get(base_url.format(2))

In [7]:
soup = bs4.BeautifulSoup(res.text, 'lxml')

In [8]:
tv = soup.select(".column")

In [9]:
example = tv[1]

In [10]:
example

<li class="column">
<script type="text/javascript">
var customTextOptionMap = {};
</script>
<!-- product_light_cache_view -->
<div class="columnContent" data-ctgid="12" data-position="30" data-searchcount="2185" id="p-409750698">
<div class="pro">
<a class="plink" data-id="409750698" href="https://www.n11.com/urun/navigold-ng-1930-19-hd-led-tv-monitor-2297438?magaza=naviprime" title='Navigold NG-1930 19" HD Led TV &amp; Monitör'>
<img alt='Navigold NG-1930 19" HD Led TV &amp; Monitör' class="lazy" data-original="https://n11scdn.akamaized.net/a1/140/elektronik/led-tv/navigold-ng-1930-19-hd-led-tv-monitor__0658884723468465.jpg" height="140" src="https://n11scdn.akamaized.net/static/css/jquery/img/blank.gif" width="140"/>
<h3 class="productName">
                                Navigold NG-1930 19" HD Led TV &amp; Monitör
            </h3>
<span class="loading"></span>
</a>
<div class="ratingCont">
<span class="rating r90"></span>
<span class="ratingText">(52)</span>
</div>
<span class="t

In [ ]:
# How to find name, seller and price information

In [11]:
example.select(".plink")[0]['title']

'Navigold NG-1930 19" HD Led TV & Monitör'

In [12]:
example.select(".sallerInfo.cPoint")[0]['title']

'naviprime'

In [13]:
example.select(".productDisplayPrice")[0]['value']

'1,377.5'

In [14]:
# Build a List for all Tv's

In [15]:
tvs = []

for n in range(2,51):
    
    scrape_url = base_url.format(n)
    res = requests.get(scrape_url)
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    
    tv = soup.select(".column")
    tvs.append(tv)

In [16]:
# Get the name, seller and price information from tvs

In [17]:
Name = []
Seller = []
Price = []

for i in range(0,49):
    for j in range(0,25):
        tv_name = tvs[i][j].select(".plink")[0]['title']
        Name.append(tv_name)
        tv_seller = tvs[i][j].select(".sallerInfo.cPoint")[0]['title']
        Seller.append(tv_seller)
        tv_price = tvs[i][j].select(".productDisplayPrice")[0]['value']
        Price.append(tv_price)

In [18]:
len(Price)

1225

In [19]:
# we have 1225 tv.

In [20]:
# Converting List to DataFrame

In [21]:
df = pd.DataFrame({'Name': Name, 'Seller': Seller, 'Price': Price})

In [22]:
# Converting price type from object to float

In [23]:
for i in range(len(df)):
    df['Price'][i] = re.sub(r",", "", df['Price'][i], flags=re.I)

In [24]:
df['Price'] = df['Price'].astype(float)

In [25]:
# Extract name, size and smartness information from Name column.

In [26]:
Brand = []

for i in range(len(df)):
    Brand.append(df['Name'][i].split(' ')[0])

In [27]:
df['Brand'] = Brand

In [28]:
Model = []

for i in range(len(df)):
    Model.append(df['Name'][i].split(' ')[1])

In [29]:
df['Model'] = Model

In [68]:
# There are not any model name below 3 characters.

In [30]:
for i in range(len(df)):
    if len(df['Model'][i]) < 3:
        df['Model'][i] = np.NaN

C:\Users\ahmet\AppData\Local\Temp/ipykernel_19092/1808185688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Model'][i] = np.NaN


In [31]:
Size = []

for i in range(len(df)):
    Size.append(re.findall(r"\d+\"", df['Name'][i]))

In [32]:
df['Size'] = Size

In [33]:
df['Size'] = df['Size'].str[0]

In [34]:
df['Size'] = df['Size'].replace(regex=r'\"', value='')

In [35]:
Smart = []

for i in range(len(df)):
    Smart.append(re.findall(r"Smart", df['Name'][i]))

In [36]:
df['Smart'] = Smart

In [37]:
df['Smart'] = df['Smart'].str[0]

In [38]:
df.drop('Name',axis=1, inplace=True)

In [39]:
# Setting column's order

In [40]:
order = ['Brand', 'Model', 'Size', 'Smart', 'Seller', 'Price']

In [41]:
df = df.reindex(order, axis=1)

In [63]:
df

,Brand,Model,Size,Smart,Seller,Price
0,Next,YE-40020D2,40,0,DALOGLUDİGİTAL,2702.00
1,Navigold,NG-1930,19,0,naviprime,1377.50
3,TCL,55C728,55,1,Comfor,14399.04
4,Vestel,32FA9500,32,1,OrhanMob-Vestel-YOB,3087.04
5,Woon,WN49DAL08/0202,49,0,DijiSepet,4052.04
...,...,...,...,...,...,...
1220,Regal,55R753U,55,1,RegalKurumsalMağaza,7839.02
1221,Vestel,43Q9900,43,1,VestelKurumsalMağaza,8973.54
1222,Vestel,43F9510,43,1,VestelKurumsalMağaza,6049.00
1223,Vestel,55UA9600,55,1,VestelKurumsalMağaza,8329.02


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1115 entries, 0 to 1224
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Brand   1115 non-null   object 
 1   Model   1115 non-null   object 
 2   Size    1115 non-null   int32  
 3   Smart   1115 non-null   int64  
 4   Seller  1115 non-null   object 
 5   Price   1115 non-null   float64
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 56.6+ KB


In [ ]:
# If Smart is NaN this means tv is not smart TV. Smart tv = 1, Not Smart tv = 0.

In [49]:
df['Smart'] = df['Smart'].fillna(0)

In [50]:
df['Smart'] = df['Smart'].replace('Smart',1)

In [ ]:
# Droping missing values

In [54]:
df.dropna(inplace=True)

In [ ]:
# Changing type of 'Size'

In [61]:
df['Size'] = df['Size'].astype(int)

In [66]:
df.to_csv('n11_tv.csv',index=False)